### Import data

In [ ]:
include("importData.jl")
include("Hill.jl")
include("plot.jl")

# import drug concentrations
param_lap_dde = CSV.read(".//figures//Dox//params_dox_DDE.csv")
concentrations = permutedims(Vector(param_lap_dde[8,2:end]));
print(concentrations)

# import G1 and G2 data
pop, g2, g1, g2_0, g1_0 = get_data("..//data//dox.csv", "..//data//dox_pop.csv");

### The new approach with the BlackBoxOptim; optimizing multiple objective functions (BorgMOEA)

In [ ]:
hill(p, concentration) =  p[2] + ((p[3] - p[2]) / (1 + 10^((concentration - p[1])*p[4])))

function residHill(hillParams, concentrations, g1, g2, g1_0, g2_0)
    """ This functions takes in hill parameters for all the concentrations and calculates
    DDE parameters, passes them to residual function and based off of these, optimizes the model
    and estimates hill parameters. """

    num_concentration = length(concentrations)
    data_size = length(g1[:, 1])
    obj = zeros(num_concentration)

#     EC50  for all of the trials is hillParams[1]
    for ii in 1:num_concentration
        alpha = hill(append!([hillParams[1]], hillParams[2:4]), concentrations[ii])
        beta = hill(append!([hillParams[1]], hillParams[5:7]), concentrations[ii])
        tau1 = hill(append!([hillParams[1]], hillParams[8:10]), concentrations[ii])
        tau2 = hill(append!([hillParams[1]], hillParams[11:13]), concentrations[ii])
        gamma1 = hill(append!([hillParams[1], hillParams[14]], [0, hillParams[15]]), concentrations[ii])
        gamma2 = hill(append!([hillParams[1], hillParams[16]], [0, hillParams[17]]), concentrations[ii])

        pp = [alpha, beta, tau1, tau2, gamma1, gamma2]
        _, obj = optimization(g1, g2, g1_0, g2_0, pp, ii)
        
    end
    return obj
end

param_gem_dde = CSV.read(joinpath(".", "figures", "Gem", "params_gem_DDE.csv"))
concentrations = permutedims(Vector(param_gem_dde[8,2:end]));
guess = log.([100.0, 0.005, 0.3, 0.02, 0.02, 0.006, 0.02, 25.0, 1.2, 0.02, 20.0, 0.2, 0.02, 0.001, 0.02, 0.01, 0.02])
residHill(guess, concentrations, g1, g2, g1_0, g2_0)

In [ ]:
function optimize_hill(concentrations, guess, low, high, g1, g2)
    obj =residHill(guess, concentrations, g1, g2)
     results_dde = optimize(obj, lower=low,
                           upper=high,
                           log.(guess),
                           Fminbox(Optim.LBFGS()))
    return Optim.minimizer(results_dde)
end

In [ ]:
# lower bound
low = log.([50.0, 0.001, 0.2, 0.01, 0.01, 0.001, 0.01, 20.0, 0.1, 0.01, 19.0, 0.01, 0.01, 0.001, 0.01, 0.001, 0.01])
# upper bound
high = log.([250.0, 0.01, 0.4, 0.04, 0.05, 0.02, 0.04, 35.0, 3.0, 0.04, 23.0, 2.0, 0.4, 0.01, 0.04, 0.04, 0.04])
# initial guess

guess = log.([100.0, 0.005, 0.3, 0.02, 0.02, 0.006, 0.02, 25.0, 1.2, 0.02, 20.0, 0.2, 0.02, 0.001, 0.02, 0.01, 0.02])
# print(guess)
optimize_hill(concentrations, guess, low, high, g1, g2)

### Hill model (the former one), residual functions and optimization

In [ ]:
# lower bound
low = [50.0, 0.001, 0.2, 0.01, 0.01, 0.001, 0.01, 20.0, 0.1, 0.01, 19.0, 0.01, 0.01, 0.001, 0.01, 0.001, 0.01]
# upper bound
high = [250.0, 0.01, 0.4, 0.04, 0.05, 0.02, 0.04, 35.0, 3.0, 0.04, 23.0, 2.0, 0.4, 0.01, 0.04, 0.04, 0.04]
# initial guess
guess = [100.0, 0.005, 0.3, 0.02, 0.02, 0.006, 0.02, 25.0, 1.2, 0.02, 20.0, 0.2, 0.02, 0.001, 0.02, 0.01, 0.02]

optimizer_result = optimize_hill(concentrations, guess, low, high)

### Plot Hill curve for all of the DDE model parameters

In [ ]:
using Plots;
params = optimizer_result.minimizer
effects = zeros(6, 8)
for i in 1:8
    effects[1, i] = hill(params[1:4], concentrations[i])
    effects[2, i] = hill(append!([params[1]], params[5:7]), concentrations[i])
    effects[3, i] = hill(append!([params[1]], params[8:10]), concentrations[i])
    effects[4, i] = hill(append!([params[1]], params[11:13]), concentrations[i])
    effects[5, i] = hill(append!([params[1], params[14]], [0, params[15]]), concentrations[i])
    effects[6, i] = hill(append!([params[1], params[16]], [0, params[17]]), concentrations[i])
end


c = permutedims(Array(concentrations))
plot(c, effects[1, :], label = "alpha", linewidth = 2, xlabel = "concentration [nM]", ylabel = "drug effect")
plot!(c, effects[2, :], label = "beta", linewidth = 2, legend =:right)
plot!(c, effects[5, :], label = "gamma1", linewidth = 2)
plot!(c, effects[6, :], label = "gamma2", linewidth = 2)


In [ ]:
plot(c, effects[3, :], label = "tau1", linewidth = 2, xlabel = "concentration [nM]", ylabel = "drug effect")
plot!(c, effects[4, :], label = "tau2", linewidth = 2, legend =:right)

### Plot the data with the new set of parameters for DDE

In [ ]:
j = 6 # trial number
# for instance we want to plot the data and estimated for some trial 
alpha_ = hill(params[1:4], concentrations[j])
beta_ = hill(append!([params[1]], params[5:7]), concentrations[j])
tau1_ = hill(append!([params[1]], params[8:10]), concentrations[j])
tau2_ =hill(append!([params[1]], params[11:13]), concentrations[j])
gamma1_ = hill(append!([params[1], params[14]], [0, params[15]]), concentrations[j])
gamma2_ = hill(append!([params[1], params[16]], [0, params[17]]), concentrations[j])

dde_params = [alpha_, beta_, tau1_, tau2_, gamma1_, gamma2_]
plotIt(dde_params, g1, g2, g1_0, g2_0, pop, j, "Lapatinib")


In [1]:
using DelayDiffEq, DiffEqParamEstim, BlackBoxOptim, DataFrames, LsqFit
using Plots
gr()

include("importData.jl")
include("plot.jl")

# import data from the path, in which:
# pop: population data
# g1, g2: g1 and g2 data
# initial: initial number of cells in g1 and in g2 at time 0
pop, g2, g1, g2_0, g1_0 = get_data(joinpath("..", "data", "lap.csv"),
                                   joinpath("..", "data", "lap_pop.csv"))

# time points
times = range(0.0; stop = 95.5, length = 192)

# model
function DDEmodel(du, u, h, p, t)
    du[1] = -p[1]*(h(p, t-p[3])[1]) + 2*p[2]*(h(p, t-p[4])[2]) - p[5]*u[1]
    du[2] = p[1]*(h(p, t-p[3])[1]) - p[2]*(h(p, t-p[4])[2]) - p[6]*u[2]
end

# estimate history function
exp_model(t, p) = @. p[1] * exp(t * p[2]) # exponential model
fit1 = curve_fit(exp_model, times, g1[:, 1], [1.0, 0.5])
fit2 = curve_fit(exp_model, times, g2[:, 1], [1.0, 0.5])
h(p, t) = [exp_model(t, coef(fit1)); exp_model(t, coef(fit2))]

# initial guess
initial_guess = [0.02798, 0.025502, 21.3481, 10.2881, 0.0001, 0.0001]

# problem
prob = DDEProblem(DDEmodel, [g1_0[3], g2_0[3]], h, extrema(times), initial_guess;
                  constant_lags = [initial_guess[3], initial_guess[4]])

# DDE algorithm:
# switch automatically between non-stiff solver Tsit5 and stiff solver Rosenbrock23
alg = MethodOfSteps(AutoTsit5(Rosenbrock23()))

# plot initial guess
plot(solve(prob, alg), label = ["u1(t) (guess)", "u2(t) (guess)"], legend = :topleft,
     show = true)

# plot data
data = vcat(g1[:, 3]', g2[:, 3]')
scatter!(times, data', show = true)

# define problem generator (optimization in log space)
function prob_generator(prob, p)
    exp_p = exp.(p)
    remake(prob; p = exp_p, constant_lags = [exp_p[3], exp_p[4]])
end

# define objective function (L2 loss)
obj = build_loss_objective(prob, alg, L2Loss(times, data);
                           prob_generator = prob_generator)

# perform global optimization
results_dde = bboptimize(obj;
                         SearchRange = [(-6.0, 6.0), (-6.0, 6.0), (0.0, 6.0),
                                        (0.0, 6.0), (-10.0, 0.0), (-10.0, 0.0)],
                         NumDimensions = 6)

# plot solution with best candidate
min_p = exp.(best_candidate(results_dde))

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = get_data(::String, ::String) at importData.jl:30
└ @ Main /home/farnazm/forwardDiff/one_state_model/model/importData.jl:30


Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64},RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},RandomBound{RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.52 secs, 41 evals, 23 steps, improv/step: 0.435 (last = 0.4348), fitness=82431.338382857
1.03 secs, 113 evals, 65 steps, improv/step: 0.446 (last = 0.4524), fitness=9096.789211336
1.54 secs, 197 evals, 127 steps, improv/step: 0.480 (last = 0.5161), fitness=9096.789211336
2.45 secs, 241 evals, 163 steps, improv/step: 0.460 (last = 0.3889), fitness=9096.789211336
3.88 secs, 308 evals, 221 steps, improv/step: 0.448 (last = 0.4138), fitness=9096.789211336
4.38 secs, 436 evals, 337 steps, improv/step: 0.415 (last = 0.3534), fitness=4972.655182915
4.88 secs, 585 evals, 482 steps, improv/step: 0.361 (last = 0.2345), fitness=4972.655182915
5.38 secs, 726 evals, 623 steps, improv/step: 0.335 (last = 0.2482), fitness=4972.655182915
5.89 secs, 868 evals, 765 steps, improv/step: 0.328 (last = 0.2958),

6-element Array{Float64,1}:
 0.03894797319908397 
 0.025745628461301292
 1.0000495348935614  
 1.000100863596819   
 4.573225956565245e-5
 4.553391072695829e-5

In [ ]:
obj.cost_function.verbose_opt